In [10]:
import pandas as pd
import os
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
#from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

#commande à n'exécuter qu'une fois
#os.chdir('data/')

In [11]:
#on prend la liste des fichiers
files = [f for f in os.listdir('.') if os.path.isfile(f)]
data = pd.read_csv(files[0])
#la colonne +/- n'est pas prise en complte car on a pas sa moyenne pour les matchs de Wembenyama
data = data.loc[:, data.columns!='+/-']
#convertion des minutes par match en flottant
data.loc[:,'MP'] = [float(x.split(':')[0])+ 0.006*float(x.split(':')[1]) for x in data.loc[:,'MP']]
#concatenation de toutes les tables csv en un seul tableau
for file in files[1:]:
    match = pd.read_csv(file)
    match = match.loc[:,match.columns!='+/-']
    match.loc[:,'MP'] = [float(x.split(':')[0])+ 0.006*float(x.split(':')[1]) for x in match.loc[:,'MP']]
    data = pd.concat([data,match])
data.reset_index(drop=True, inplace=True)

print("nombre d'individus: ", len(data.index))

nombre d'individus:  223


In [12]:
#imputation des données manquantes
#remplace les données absentes par une données de la même colonne, avec la même valeur pour la variable Résultat
data_win = data[data['Résultat'] == 1]
data_lose = data[data['Résultat'] != 1]
for col in data_win.columns:
    for i,rows in data_win.iterrows():
        if pd.isna(data_win.loc[i,col]):
            data_win.loc[i,col] = np.random.choice(data_win[data_win[col].notna()].loc[:,col])
for col in data_lose.columns:
    for i,rows in data_lose.iterrows():
        if pd.isna(data_lose.loc[i,col]):
            data_lose.loc[i,col] = np.random.choice(data_lose[data_lose[col].notna()].loc[:,col])
data = pd.concat([data_win,data_lose])

X_train = data.loc[:, data.columns!='Résultat']
X_train = X_train.iloc[:,1:]
Y = data.loc[:,'Résultat']
VW = np.array([30.2 ,6.9 ,15.9 ,0.435 ,1.4 ,4.9 ,0.278 ,3.3 ,4.2 ,0.774 ,2.3 ,8.4 ,10.7 ,2.8 ,1.4 ,3.0 ,3.3 ,2.5 ,18.5])

In [13]:
#Standardisation
scaler = sk.preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)

In [14]:
#régression linéaire
reg = LinearRegression().fit(X_scaled, Y)
for k in range(len(reg.coef_)):
    print('coef de ',data.columns[k], ':', reg.coef_[k])
print('constante: ',reg.intercept_)

coef de  Unnamed: 0 : -0.0625328483730836
coef de  MP : 0.10657002570263815
coef de  FG : -0.14345339199124466
coef de  FGA : -0.04286816482726524
coef de  FG% : 0.14365343380078707
coef de  3P : -0.08139585349260567
coef de  3PA : -0.06690390153742647
coef de  3P% : -0.30486236419921153
coef de  FT : 0.286212809304426
coef de  FTA : 0.2611929013007886
coef de  FT% : -0.01719680368100993
coef de  ORB : 0.036889322667084036
coef de  DRB : 0.021170008119222983
coef de  TRB : 0.025254772536418543
coef de  AST : 0.013604339727185692
coef de  STL : 0.014538592579304208
coef de  BLK : -0.018549194476233967
coef de  TOV : 0.04562496778797842
coef de  PF : 0.02485773358715399
constante:  0.5112107623318386


In [15]:
#régression logistique
clf = LogisticRegression(random_state=0).fit(X_scaled, Y)
VW = VW.reshape(1, -1)
print(np.shape(VW))
print('prédiction: ', clf.predict(VW))
print('proba de perdre: ', clf.predict_proba(VW)[0][0])
print('proba de gagner: ', clf.predict_proba(VW)[0][1])

(1, 19)
prédiction:  [0]
proba de perdre:  0.999850053066774
proba de gagner:  0.00014994693322600432


In [16]:
#arbre de décision
clf = tree.DecisionTreeClassifier(max_depth=6)
clf = clf.fit(X_scaled, Y)
print('Les résultats suivants ne sont pas consistants suivant la séparation entrainement/vérif')
print('prédiction: ', clf.predict(VW))
print('proba de perdre: ', clf.predict_proba(VW)[0][0])
print('proba de gagner: ', clf.predict_proba(VW)[0][1])

Les résultats suivants ne sont pas consistants suivant la séparation entrainement/vérif
prédiction:  [1]
proba de perdre:  0.0
proba de gagner:  1.0


In [17]:
#RandomForest
clf = RandomForestClassifier(max_depth=2)
clf.fit(X_scaled, Y)
print('prédiction: ', clf.predict(VW))
print('proba de perdre: ', clf.predict_proba(VW)[0][0])
print('proba de gagner: ', clf.predict_proba(VW)[0][1])

prédiction:  [1]
proba de perdre:  0.3287391598670345
proba de gagner:  0.6712608401329656
